In [ ]:
import os
import numpy as np
# import cupy as cp
import scipy.sparse as ssp
from PIL import Image
import admm

In [ ]:
# DATA_PATH = '../../../OneDrive - m.titech.ac.jp/Lab/data'
DATA_PATH = '../data'
OBJ_NAME = "Cameraman"
# H_SETTING = "FISTA_p-5_lmd-100_m-255"
H_SETTING = "gf"
CAP_DATE = "241114"
EXP_DATE = "241118"
n = 128
m = 255

In [ ]:
# def createDrgb(n):
#     I = ssp.eye(n**2, format="lil")
#     Dx = I - ssp.lil_matrix(np.roll(I.toarray(), 1, axis=1))
#     Dx[n - 1 :: n, :] = 0
#     Dy = I - ssp.lil_matrix(np.roll(I.toarray(), n, axis=1))
#     Dy[-n:, :] = 0
#     D0 = ssp.lil_matrix((n**2, n**2))
#     D = ssp.block_array([[Dx, D0, D0], [D0, Dx, D0], [D0, D0, Dx], [Dy, D0, D0], [D0, Dy, D0], [D0, D0, Dy]])
#     return D

def create_D_mono(n):
    I = ssp.eye(n**2, format="lil")

    Dx = I - ssp.lil_matrix(np.roll(I.toarray(), 1, axis=1))
    Dx[n - 1 :: n, :] = 0
    Dy = I - ssp.lil_matrix(np.roll(I.toarray(), n, axis=1))
    Dy[-n:, :] = 0

    D = ssp.vstack([Dx, Dy])

    return D


D = create_D_mono(n)

In [ ]:
captured = Image.open(f"{DATA_PATH}/capture_{CAP_DATE}/{OBJ_NAME}.png").convert("L")
captured = np.array(captured)
g = captured.ravel()

In [ ]:
PREFIX = ""
# PREFIX = "int_"
H = np.load(f"{DATA_PATH}/{EXP_DATE}/systemMatrix/H_matrix_{PREFIX}{H_SETTING}.npy")
print("H shape:", H.shape, "type(H):", type(H), "H.dtype:", H.dtype)

In [ ]:
H_sparse = ssp.csr_matrix(H)
del H

In [ ]:
admm = admm.Admm(H_sparse, g, D)
f, err = admm.solve()

In [ ]:
f = np.clip(f, 0, 1) * 255
F = f.reshape(n, n)
F = F.astype(np.uint8)
F_image = Image.fromarray(F)
tau = np.log10(admm.tau)
mu1 = np.log10(admm.mu1)
mu2 = np.log10(admm.mu2)
mu3 = np.log10(admm.mu3)
if not os.path.exists(f"{DATA_PATH}/{EXP_DATE}/reconst"):
    os.makedirs(f"{DATA_PATH}/{EXP_DATE}/reconst")
SAVE_PATH = f"{DATA_PATH}/{EXP_DATE}/reconst/{OBJ_NAME}_{H_SETTING}_admm_t-{tau}_m{mu1}m{mu2}m{mu3}.png"
F_image.save(SAVE_PATH, format="PNG")
print(SAVE_PATH)